# 대신증권 모의투자 환경
---
> 본격적으로 API를 통해 가상계좌에서 매수를 수행해보겠습니다. 이를 위해 먼저 PyCharm과 같은 에디터를 관리자 모드로 실행해야 합니다. 참고로 매수/매도 API를 이용하려면 주식 시장의 운영 시간에 코드를 실행해야 합니다. 대신증권의 모의투자는 가상이긴 하지만 실제 시장의 운영 시간과 같은 시간에만 운영되기 때문입니다.

> 주말, 공휴일 주식 시장 휴장
* [API 자료](https://money2.daishin.com/e5/mboard/ptype_basic/plusPDS/DW_Basic_List.aspx?boardseq=299&m=9508&p=8831&v=8638)

In [1]:
import win32com.client

## 모의투자 환경에서 매매
* CpTdUtil 클래스는 주문 오브젝트를 사용하기 위해 필요한 초기화 과정들을 수행한다
* 매매하기 위해서 0311 모듈 사용
* 0311 모듈은 RQ/RP 방식
* 그러나 결과를 getDataValue 못쓰고 대신 conclusion 클래스로부터 얻는다.
* 모의투자인 경우 TradeInit 호출시 사용되는 서비스(계좌비밀번호 검증하는 서비스)가 제공되고있지 않습니다.
* 반드시 먼저 주문 오브젝트 사용 동의를 해야한다.
* [1846] 시스템트레이딩 이용 신청/해지’화면을 통해 시스템트레이딩이용신청
* [1829] HTS 이중접속서비스 신청/해지’ 화면을 통해 이중접속신청

In [2]:
cpTdUtil = win32com.client.Dispatch('CpTrade.CpTdUtil')
cpTd0311 = win32com.client.Dispatch('CpTrade.CpTd0311')
cpTdUtil.TradeInit() # 반드시 초기화 해줘야함 - 정상이면 0 반환, 주식장 열려있는 시간에만 정상적으로 창이 뜬다.

com_error: (-2147352567, '예외가 발생했습니다.', (0, None, 'U-CYBOS가 서버에 접속되어 있지 않습니다.', None, 0, -2147483638), None)

## 테스트
---
* 증거금이 부족하다? - 
* 상품관리구분을 확인해라? - 3 번째 데이터로 상품구분을 넣어줘야함
* 4396 모의투자 IOC/FOK는 주문불가하다? - 최유리지정가 최우선지정가를 뜻하는데 모투는 안됨?
---
* 주문하면 한 번에 안사질수도 있다. 지정가로 하면 더욱 그럼 물량이 없을수도
* API 로 확인하는 방법 또는 HTS 실시간 주문체결 내역조회 탭에서

In [18]:
# 현금 매수
def buy():
    account_num = cpTdUtil.AccountNumber[0]
    accFlag = cpTdUtil.GoodsList(account_num, 1)  # 주식상품 구분
    print(account_num, accFlag[0])
    cpTd0311.SetInputValue(0, 2) # 첫번째인자 : 주문종류코드, 두번째인자 : 매도(1)/매수(2)
    cpTd0311.SetInputValue(1, account_num) # 계좌, 계좌번호 값
    cpTd0311.SetInputValue(2, accFlag[0])   # 상품구분 - 주식 상품 중 첫번째
    cpTd0311.SetInputValue(3, 'A003540') # 종목코드, 코드 값 (대신증권 주식)
    cpTd0311.SetInputValue(4, 10) # 주문 수량
    cpTd0311.SetInputValue(5, 13100) # 주문 단가
    cpTd0311.SetInputValue(7, 0)   # 주문 조건 구분 코드, 0: 기본 1: IOC 2:FOK
    cpTd0311.SetInputValue(8, "01")   # 주문호가 구분코드 - 01: 보통

    # 요청
    cpTd0311.BlockRequest() # 

    # 확인은 conclusion 클래스로
    if cpTd0311.GetDibStatus() != 0:
        print("통신상태", cpTd0311.GetDibStatus(), cpTd0311.GetDibMsg1())
    else:
        print("예약주문 성공, 예약번호 #")

In [19]:
# 현금 매도
def sell():
    account_num = cpTdUtil.AccountNumber[0]
    accFlag = cpTdUtil.GoodsList(account_num, 1)  # 주식상품 구분
    print(account_num, accFlag[0])

    cpTd0311 = win32com.client.Dispatch("CpTrade.CpTd0311")
    cpTd0311.SetInputValue(0, "1")   #  1: 매도
    cpTd0311.SetInputValue(1, account_num )   #  계좌번호
    cpTd0311.SetInputValue(2, accFlag[0])   #  상품구분 - 주식 상품 중 첫번째
    cpTd0311.SetInputValue(3, "A003540")   #  종목코드 - A003540 - 대신증권 종목
    cpTd0311.SetInputValue(4, 1)   #  매도수량 10주
    cpTd0311.SetInputValue(5, 14100)   #  주문단가  - 14,100원
    cpTd0311.SetInputValue(7, "0")   #  주문 조건 구분 코드, 0: 기본 1: IOC 2:FOK
    cpTd0311.SetInputValue(8, "01")   # 주문호가 구분코드 - 01: 보통

    # 매도 주문 요청
    cpTd0311.BlockRequest()

    rqStatus = cpTd0311.GetDibStatus()
    rqRet = cpTd0311.GetDibMsg1()
    print("통신상태", rqStatus, rqRet)
    if rqStatus != 0:
        exit()

In [20]:
buy()
sell()

335021511 10
예약주문 성공, 예약번호 #
335021511 10
통신상태 0 0039 모의투자 매도 주문이 완료되었습니다.(ordss.cststkord)
